# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [1]:
!pip install "pymysql==1.0.2"

In [2]:
!pip install sqlalchemy --upgrade

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 13.0 MB/s eta 0:00:00


> #### SQL Alembic permite versionar bases de datos

> #### Pydantic: Sirve para modelar datos

In [3]:
!pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ------- -------------------------------- 3.1/16.1 MB 18.4 MB/s eta 0:00:01
   ------------------ --------------------- 7.6/16.1 MB 20.4 MB/s eta 0:00:01
   ----------------------------- ---------- 11.8/16.1 MB 20.5 MB/s eta 0:00:01
   -------------------------------------- - 15.5/16.1 MB 19.8 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 19.1 MB/s eta 0:00:00


### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [ ]:
host = "db-name-date.cneqyyuq8zjx.eu-north-1.rds.amazonaws.com"
username = "username"
password = "xxxxxx"
port = 3306

### Conexion BD

In [5]:
import pymysql

In [ ]:
'''
pymysq].cursors.Dictcursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas
'''

db = pymysql.connect(host=host,
                     user=username,
                     password=password,
                     cursorclass=pymysql.cursors.DictCursor
)
# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db. cursor()

### Version DB

In [7]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta-
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries que modifiquen la base de datos de alguna manera:
de insert de datos, creación de tablas, etc 
'''
cursor.execute("SELECT VERSION()")
version = cursor.fetchone ()
print (f"MysQL version: {version}")

MysQL version: {'VERSION()': '8.0.39'}


### Creación de DB

In [8]:
#Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor. execute(drop_db)
# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

1

In [ ]:
# Podemos eliminar la BD
#drop_db = '''DROP DATABASE country_database'''
#cursor. execute(drop_db)
# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas


1

In [10]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

### Creación de tablas

In [9]:
'''
Creamos una tabla 
Esto da error porque no le hemos dicho al servidor que BD
queremos usar
'''

# Crear una tabla que se llame country
create_table = '''
    CREATE TABLE country (
        id INT NOT NULL auto_increment,
        name TEXT,
        continent TEXT,
        population INT,
        gdp DOUBLE,
        primary key (id)
    )
'''
cursor.execute(create_table)

OperationalError: (1046, 'No database selected')

### Seleccionar la BD

In [10]:
# Para usar la BD recien creada
cursor.connection.commit()
use_db = ''' USE country_database'''
cursor.execute(use_db)

0

In [11]:
# Ahora sí se puede crear la tabla en la BBDD seleccionada
cursor.execute(create_table)

0

In [12]:
# Chequear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_country_database': 'country'}]

In [13]:
# Si queremos borrar alguna tabla 
# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)

### Insertar datos

In [15]:
insert_data = '''
INSERT INTO country(name, continent, population, gdp)
VALUES ('Spain','Europe',47, 1.74)
'''
cursor.execute(insert_data)

1

In [14]:
name = "France"
continent = "Europe"
population = 54
gdp = 2

In [16]:
insert_data = f'''
INSERT INTO country(name,continent,population,gdp) VALUES ("{name}","{continent}",{population},{gdp})'''

In [17]:
cursor.execute(insert_data)

1

### Leer datos

In [18]:
query = '''SELECT * FROM country WHERE name LIKE "%pa%"'''
cursor.execute(query)

1

In [19]:
print(cursor.fetchall())

[{'id': 1, 'name': 'Spain', 'continent': 'Europe', 'population': 47, 'gdp': 1.74}]


### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [20]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [21]:
try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    cursor.fetchall()
except Exception as e:
    print(e)

### Insertar datos de un CSV

In [22]:
# Creamos el CSV
import pandas as pd

df= pd.read_csv('country_data.csv', sep=';')

In [23]:
df

,name,continent,population,gdp
0,France,Europe,70,2.7
1,Japan,Asia,126,4.9
2,Argentina,America,44,0.5
3,Brasil,America,209,1.8


In [24]:
for datos in zip(*[valor.values() for clave, valor in df.to_dict().items()]):
    insert = "INSERT INTO country(name,continent,population,gdp) VALUES ('%s','%s',%d,%.1f)" \
    %(datos[0],datos[1],datos[2],datos[3])
    cursor.execute(insert)

In [25]:
query = "SELECT * FROM country"
cursor.execute(query)
cursor.fetchall()

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.74},
 {'id': 2,
  'name': 'France',
  'continent': 'Europe',
  'population': 54,
  'gdp': 2.0},
 {'id': 3,
  'name': 'France',
  'continent': 'Europe',
  'population': 70,
  'gdp': 2.7},
 {'id': 4,
  'name': 'Japan',
  'continent': 'Asia',
  'population': 126,
  'gdp': 4.9},
 {'id': 5,
  'name': 'Argentina',
  'continent': 'America',
  'population': 44,
  'gdp': 0.5},
 {'id': 6,
  'name': 'Brasil',
  'continent': 'America',
  'population': 209,
  'gdp': 1.8}]

In [26]:
# Utilizando sqlalchemy
from sqlalchemy import create_engine
import pymysql

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user=username, pw=password, host=host, db="country_database"))

In [27]:
df.to_sql(name="country", con=engine, if_exists="append", index=False)

4

In [28]:
db.commit()

### Cerrar la conexión cuando acabemos

In [29]:
db.close()

In [30]:
conn = engine.connect()
conn.close()